In [15]:
# 직원 순환 배치 PPO 강화학습
import datetime
import pandas as pd
import numpy as np
import torch
import gym
from gym import spaces
from stable_baselines3 import PPO
from sklearn.metrics.pairwise import euclidean_distances
import warnings
warnings.filterwarnings("ignore")

# 1. 실험 재현성을 위한 난수 시드(SEED) 고정
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

# 2. 직원 및 지점 정보를 CSV 파일에서 로드
employees = pd.read_csv("./data/rotation_employees.csv")
branches = pd.read_csv("./data/rotation_branches.csv")

# 3. 근무 기간에 따라 순환 배치 대상 직원(eligible)과 비대상 직원(non-eligible) 분류
eligible_employees = employees[employees['years_at_branch'] > 3].reset_index(drop=True)
non_eligible_employees = employees[employees['years_at_branch'] <= 3].copy()
non_eligible_employees['new_branch'] = non_eligible_employees['current_branch']
non_eligible_employees['reassigned'] = False

# 4. 강화학습 환경에서 사용할 지점명 리스트 준비
branch_names = branches['branch_name'].tolist()

# 5. 직원의 집과 각 지점 간의 유클리드 거리를 계산하는 함수 정의하고, 이를 이용해 거리 행렬 생성
def calculate_distance_matrix(emp_df, branch_df):
    home_coords = emp_df[['home_x', 'home_y']].values
    branch_coords = branch_df[['branch_x', 'branch_y']].values
    return euclidean_distances(home_coords, branch_coords)
    
distance_matrix = calculate_distance_matrix(eligible_employees, branches)

# 6. 직원 순환 배치 문제를 위한 사용자 정의 Gym 환경('RotationEnv') 정의
class RotationEnv(gym.Env):

    # 7. 환경 초기화 메서드: 필요한 데이터, 공간 정의, 상태 변수 초기화 등 수행
    def __init__(self, emp_df, branch_df, distance_matrix, full_emp_df): # ranks 파라미터 제거
        super().__init__()
        self.emp_df = emp_df  # 이동 대상 직원
        self.branch_df = branch_df # 지점 정보
        self.distance_matrix = distance_matrix # 거리 행렬

        self.n_emp = len(emp_df) # 이동 대상 직원 수
        self.n_branches = len(branch_df) # 지점 수
        self.max_branch_capacity = 12 # 지점 최대 정원
        self.min_branch_capacity = 8  # 지점 최소 정원

        # 7.1 에이전트가 배치함에 따라 변경될 지점별 인원 현황
        self.branch_counts = np.zeros(self.n_branches, dtype=int)
        self.current_idx = 0 # 현재 배치 중인 이동 대상 직원의 인덱스

        # 7.2 이동 비대상 직원(고정 인원)으로 인한 초기 지점 상태(인원수) 설정
        self.fixed_branch_counts = self._init_branch_state(full_emp_df) # _init_branch_state 반환값 변경에 따름

        # 7.3 행동 공간(Action Space) 정의: 에이전트는 각 직원에 대해 n_branches개의 지점 중 하나 선택
        self.action_space = spaces.Discrete(self.n_branches)
        
        # 7.4 관찰 공간(Observation Space) 정의: 직급 분포 정보 제거
        # 구성: [현재 직원 집 좌표(2), 지점별 수용력 비율(n_branches)]
        self.observation_space = spaces.Box(low=0, high=1, shape=(2 + self.n_branches,), dtype=np.float32)

    # 8. (내부 메서드) 전체 직원 데이터를 기반으로 고정된 초기 지점 상태(총 인원) 계산
    def _init_branch_state(self, full_df): # 반환값에서 rank_counts 제거
        counts = np.zeros(self.n_branches, dtype=int)
        for _, row in full_df.iterrows():
            idx = self.branch_df[self.branch_df['branch_name'] == row['current_branch']].index[0]
            counts[idx] += 1
        return counts # counts만 반환

    # 9. (Gym 표준 메서드) 환경의 난수 시드 설정
    def seed(self, seed=None):
        np.random.seed(seed)
        return [seed]

    # 10. (Gym 표준 메서드) 에피소드 시작 시 환경을 초기 상태로 리셋하고 첫 관찰 값 반환
    def reset(self):
        self.emp_df = self.emp_df.sample(frac=1).reset_index(drop=True) # 직원 순서 섞기
        self.current_idx = 0
        self.branch_counts[:] = 0 # 이번 에피소드 배치 현황 초기화

        return self._get_obs()

    # 11. (내부 메서드) 현재 상태에 대한 관찰 값을 생성하여 에이전트에게 제공
    def _get_obs(self):
        emp = self.emp_df.iloc[self.current_idx]
        home_xy = np.array([emp['home_x'] / 100.0, emp['home_y'] / 100.0])

        total_counts = self.fixed_branch_counts + self.branch_counts
        capacity_ratio = total_counts / self.max_branch_capacity

        return np.concatenate([home_xy, capacity_ratio], axis=0).astype(np.float32)

    # 12. (내부 메서드) 에이전트의 행동(action)에 대한 보상(reward) 계산. 직급 관련 패널티 로직 제거.
    def _calculate_reward(self, action, emp): # rank 파라미터 제거
        # 12-1 보상 점수를 매기기 위한 기초 정보를 준비
        projected_count = self.fixed_branch_counts[action] + self.branch_counts[action] + 1 #그 지점의 예상 총 인원
        total_required = self.min_branch_capacity * self.n_branches                         #모든 지점에 필요한 최소 총 인원
        assigned_so_far = self.current_idx + 1                                              #지금까지 보낸 직원 수
        progress_ratio = assigned_so_far / total_required if total_required > 0 else 0      #전체 배치 중 얼마나 진행됐는지 비율

        # 12-2 필수 이동 대상자(4년 이상 근무)가 원래 지점에 남는 경우 패널티 (배치 진행률 60% 이상 시 적용)
        moved_required_penalty = 0
        if emp['years_at_branch'] >= 4 and emp['current_branch'] == self.branch_df.iloc[action]['branch_name']:
            if progress_ratio >= 0.6:
                moved_required_penalty = 1

        # 12-3 정원 초과 & 미달 패널티 설정
        over_penalty = 0
        under_penalty = 0
        if projected_count > self.max_branch_capacity: # 정원 초과 패널티
            over_penalty = projected_count - self.max_branch_capacity
        elif projected_count < self.min_branch_capacity: # 정원 미달 패널티 (배치 진행률 60% 이상 시 적용)
            if progress_ratio >= 0.6:
                under_penalty = self.min_branch_capacity - projected_count
        
        # 12-4 최종 보상 계산: 각 패널티 항목에 가중치 부여
        reward = \
            - 1 * over_penalty  \
            - 4 * under_penalty  \
            - 2 * moved_required_penalty
        return reward

    # 13. (Gym 표준 메서드) 에이전트가 행동을 취하면, 환경 상태를 업데이트하고 다음 관찰, 보상, 종료 여부 등을 반환
    def step(self, action):
        # 13.1 현재 직원 정보 가져오고, 선택한 행동에 대한 점수 매기기
        emp = self.emp_df.iloc[self.current_idx]
        reward = self._calculate_reward(action, emp)

        # 13.2 선택된 지점에 현재 직원 배치 및 상태 업데이트
        self.branch_counts[action] += 1
        self.current_idx += 1 # 다음 직원으로 이동

        # 13.3 모든 이동 대상 직원 배치 완료 시 에피소드 종료
        done = self.current_idx >= self.n_emp  

        #13.4 다음 상황판 정보 만들고, 이번 행동의 최종 결과들 돌려주기
        obs = self._get_obs() if not done else np.zeros(self.observation_space.shape, dtype=np.float32)
        return obs, reward, done, {}

# 14. 정의한 RotationEnv 환경의 인스턴스 생성
env = RotationEnv(eligible_employees, branches, distance_matrix, full_emp_df=employees)

# 15. PPO 모델의 정책 신경망 아키텍처 및 활성화 함수 등 추가 설정 정의
policy_kwargs = dict(activation_fn=torch.nn.SiLU, 
                     net_arch=[dict(pi=[128,64], vf=[128,32])] # 정책망(pi), 가치망(vf) 구조
                    )

# 16. PPO 강화학습 모델 초기화. 주요 하이퍼파라미터(학습률, 엔트로피 계수 등) 설정
model = PPO(
    "MlpPolicy", # 다층 퍼셉트론(MLP) 기반 정책 사용
    env, 
    learning_rate=0.00005, # 학습률
    ent_coef=0.1,          # 엔트로피 계수 (탐험 장려)
    clip_range=0.2,        # PPO 클리핑 범위
    vf_coef=0.2,           # 가치 함수 손실 가중치
    policy_kwargs=policy_kwargs, # 신경망 설정
    seed=SEED,             # 재현성을 위한 시드
    verbose=1              # 학습 진행 상황 출력 레벨
)

# 17. PPO 모델 학습 시작
model.learn(total_timesteps=1_500_000, log_interval=20) # 총 학습 스텝 수, 로그 출력 간격

# 18. 학습된 모델 파일로 저장
model.save("./models/rotation_ppo_model") # 저장 파일명 변경하여 구분
print("**모델 학습 완료**")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 30         |
|    ep_rew_mean          | -22.3      |
| time/                   |            |
|    fps                  | 547        |
|    iterations           | 20         |
|    time_elapsed         | 74         |
|    total_timesteps      | 40960      |
| train/                  |            |
|    approx_kl            | 0.00571423 |
|    clip_fraction        | 0.00112    |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.22      |
|    explained_variance   | 0.336      |
|    learning_rate        | 5e-05      |
|    loss                 | 3.73       |
|    n_updates            | 190        |
|    policy_gradient_loss | -0.00674   |
|    value_loss           | 21.4       |
----------------------------------------
------------------------------------------
| 

In [26]:
# ---------------------------
# 예측 및 결과 저장
# ---------------------------
# 1. 이전에 학습하고 저장한 PPO 모델 로딩
model = PPO.load("./models/rotation_ppo_model")

# 2. 평가(예측)를 위해 직원 및 지점 정보 로드
employees = pd.read_csv("./data/rotation_employees.csv")  # 직원 정보
branches = pd.read_csv("./data/rotation_branches.csv")    # 지점 정보

# 3. PyTorch가 결정론적 알고리즘을 사용하도록 설정
torch.use_deterministic_algorithms(True)

# 4. 이동 대상 직원 목록을 특정 random_state로 재구성
eligible_employees = eligible_employees.sample(frac=1, random_state=42).reset_index(drop=True)

# 5. 평가를 위한 RotationEnv 환경 인스턴스 생성 및 환경 리셋
env_eval = RotationEnv(eligible_employees, branches, distance_matrix, full_emp_df=employees)
obs = env_eval.reset()

# 6. 각 이동 대상 직원에 대한 예측된 배치 결과를 저장할 리스트 초기화
assignments = []

while True:
    # 7. 학습된 모델을 사용하여 현재 관찰(obs)에 대한 최적의 행동(action, 즉 배치할 지점)을 예측
    action, _ = model.predict(obs, deterministic=False) 
    
    # 8. 예측된 행동(지점 인덱스) 출력
    print(action, end=', ')
    
    # 9. 예측된 행동을 assignments 리스트 추가
    assignments.append(int(action))
    
    # 10. 환경에서 해당 행동을 실행하고, 다음 관찰(obs), 보상(_), 종료 여부(done) 등의 정보 수신
    obs, _, done, _ = env_eval.step(action)
    
    # 11. 모든 직원이 배치되었으면(done=True) 루프 종료
    if done:
        break

# 12. 이동 대상 직원(eligible_employees) DataFrame에 예측된 'new_branch'(새 지점) 정보 추가
eligible_employees_eval['new_branch'] = [branch_names[a] for a in assignments]

# 13. 이동 대상 직원들은 재배치되었음을 표시
eligible_employees_eval['reassigned'] = True

# 14. 이동 대상 직원 결과와 비대상 직원 정보를 합쳐 최종 배치 DataFrame 생성하고, 직원 ID 순으로 정렬
final_df = pd.concat([eligible_employees_eval, non_eligible_employees], ignore_index=True)\
.sort_values(by='employee_id')

# 15. 현재 날짜와 시간을 포함한 파일명으로 최종 배치 결과를 CSV 파일로 저장
filename = f"./data/rotation_assignments_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
final_df.to_csv(filename, index=False, encoding='utf-8-sig') # UTF-8(BOM) 인코딩으로 Excel 호환성 향상

print(f"*최종 발령표 저장 완료: {filename}")

# ---------------------------
# 결과 점검 출력
# ---------------------------
# 16. 순환 배치 전(Old) 지점별, 직급별 인원 현황을 집계하여 출력
branch_summary_old = final_df.groupby(['current_branch', 'rank']).size().unstack(fill_value=0)
branch_summary_old['합계'] = branch_summary_old.sum(axis=1)
print("*지점별 인원배분 점검 Old (직급별 + 총합)")
print(branch_summary_old)

# 17. 순환 배치 후(New) 지점별, 직급별 인원 현황을 집계하여 출력
branch_summary_new = final_df.groupby(['new_branch', 'rank']).size().unstack(fill_value=0)
branch_summary_new['합계'] = branch_summary_new.sum(axis=1)
print("*지점별 인원배분 점검 New (직급별 + 총합)")
print(branch_summary_new)

moved_4yrs_eligible_ids = eligible_employees_eval['employee_id']
moved_4yrs_check_df = final_df[(final_df['employee_id'].\
                                isin(moved_4yrs_eligible_ids)) & (final_df['years_at_branch'] >= 4)].copy()

if not moved_4yrs_check_df.empty:
    # 18. 이동 대상이면서 4년 이상 근무한 직원들의 실제 이동 여부 분석
    moved_4yrs_check_df['moved'] = moved_4yrs_check_df['current_branch'] != moved_4yrs_check_df['new_branch']
    moved_summary = moved_4yrs_check_df['moved'].value_counts(dropna=False)\
    .rename(index={True: '이동함', False: '이동 안함', np.nan: '해당없음'})
    print("*4년 이상 근무한 '이동 대상' 직원의 부서 이동 여부:")
    print(moved_summary)

    # 19. 만약 이동해야 하는 4년 이상 근무자 중 이동하지 않은 인원이 있다면, 해당 직원 정보 출력
    not_moved_list = moved_4yrs_check_df[moved_4yrs_check_df['moved'] == False]
    if not not_moved_list.empty:
        print("*이동하지 않은 4년 이상 근무자 (이동 대상이었던 직원):")
        print(not_moved_list[['employee_id', 'current_branch', 'new_branch', 'years_at_branch', 'rank']])
    else:
        print("*모든 4년 이상 '이동 대상' 근무자는 부서 이동 완료 또는 대상자 없음")
else:
    print("*4년 이상 근무한 '이동 대상' 직원이 없습니다.")

4, 0, 5, 7, 8, 1, 8, 5, 2, 0, 4, 2, 8, 7, 5, 6, 7, 3, 8, 7, 2, 4, 7, 8, 3, 5, 2, 0, 8, 7, *최종 발령표 저장 완료: ./data/rotation_assignments_20250519_185305.csv
*지점별 인원배분 점검 Old (직급별 + 총합)
rank            과장  대리  사원  합계
current_branch                
Branch_0         4   3   3  10
Branch_1         4   4   3  11
Branch_2         4   3   3  10
Branch_3         4   4   3  11
Branch_4         4   3   3  10
Branch_5         3   3   3   9
Branch_6         4   4   3  11
Branch_7         3   3   2   8
Branch_8         3   3   2   8
Branch_9         4   4   4  12
*지점별 인원배분 점검 New (직급별 + 총합)
rank        과장  대리  사원  합계
new_branch                
Branch_0     3   2   4   9
Branch_1     5   2   0   7
Branch_2     3   4   4  11
Branch_3     4   5   3  12
Branch_4     2   5   3  10
Branch_5     3   5   2  10
Branch_6     2   3   3   8
Branch_7     6   3   3  12
Branch_8     5   3   4  12
Branch_9     4   2   3   9
*4년 이상 근무한 '이동 대상' 직원의 부서 이동 여부:
moved
이동함      25
이동 안함     5
Name: count, dtype: int64
*이동하지 

In [22]:
# 1. 지점의 문자열 명칭과 해당 지점 데이터의 숫자 인덱스 간의 상호 참조 가능한 매핑 구조 생성
branch_name_to_idx = {name: idx for idx, name in enumerate(branch_names)}

# 2. 순환 배치 적용 이전과 이후, 각 직원의 자택과 해당 근무 지점 간의 유클리드 거리를
# 개별적으로 저장하기 위한 리스트 변수를 초기화
past_distances, current_distances = [], []

# 3. 최종적으로 확정된 직원 배치 결과('final_df')를 한 명씩 순회하면서,
# 각 직원의 배치 전후 근무지와 자택 간의 거리 변화를 정량적으로 산출
for _, row in final_df.iterrows():
    
    # 3-1. 현재 순회 중인 직원의 자택 위치 좌표(x, y)를 NumPy 배열 형태로 추출
    home = np.array([row['home_x'], row['home_y']])
    
    # 3-2. 해당 직원의 순환 배치 이전 근무 지점명과 새로 배정된 지점명을 사용하여,
    # 앞서 생성한 'branch_name_to_idx' 매핑 테이블을 통해 각 지점의 고유 숫자 인덱스 조회
    past_idx = branch_name_to_idx[row['current_branch']]
    curr_idx = branch_name_to_idx[row['new_branch']]
    
    # 3-3. 조회된 숫자 인덱스를 활용하여 전체 지점 정보가 담긴 'branches' DataFrame에서
    # 해당 지점들의 물리적 위치 좌표(x, y) 추출
    past_loc = branches.iloc[past_idx][['branch_x', 'branch_y']].values
    curr_loc = branches.iloc[curr_idx][['branch_x', 'branch_y']].values

    # 3-4. 직원의 자택 좌표와 각 근무 지점(순환 배치 이전 및 현재)의 좌표 간의
    # 직선 거리, 즉 유클리드 거리를 'np.linalg.norm' 함수를 이용하여 계산
    # 이 함수는 두 벡터 차의 L2 노름(norm)을 계산함으로써 두 점 사이의 거리 나타냄
    past_distances.append(np.linalg.norm(home - past_loc))
    current_distances.append(np.linalg.norm(home - curr_loc))

# 4. 전 직원을 대상으로 산출된, 순환 배치 이전/이후 근무 지점까지의 개별 거리들의 산술 평균 계산
past_mean = np.mean(past_distances)
curr_mean = np.mean(current_distances)

# 5. 두 평균 거리 값의 차이를 계산하여, 이번 순환 배치가 전체 직원의 평균적인
# 자택-근무지 간 거리에 어떠한 변화를 야기했는지를 정량적으로 나타냄
diff = curr_mean - past_mean

print("\n과거 지점과 현재 배정 지점 간 거리 평균 비교:")
print(f"*과거 지점 평균 거리: {past_mean:.2f}")
print(f"*현재 배정 지점 평균 거리: {curr_mean:.2f}")

if diff < 0:
    print(f"*평균적으로 직원들의 출퇴근 거리가 {abs(diff):.2f}만큼 줄었습니다.")
else:
    print(f"*평균적으로 직원들의 출퇴근 거리가 {diff:.2f}만큼 늘었습니다.")


과거 지점과 현재 배정 지점 간 거리 평균 비교:
*과거 지점 평균 거리: 49.07
*현재 배정 지점 평균 거리: 47.75
*평균적으로 직원들의 출퇴근 거리가 1.32만큼 줄었습니다.


In [28]:
'''
각 지점 인원: 8~12명 사이
4년 이상 근무자의 전원 이동
평균 거리 감소 폭 ≥ 1 이상
'''
# 1. 최대 반복 시도 횟수 설정 및 성공 여부 플래그 초기화
max_attempts = 1000
success = False

for attempt in range(1, max_attempts + 1):
    print(attempt, end=", ") # 현재 시도 횟수 출력

    # 2. 각 시도마다 이동 대상 직원의 처리 순서를 무작위로 변경
    np.random.seed(None) # Pandas sample의 무작위성 확보
    eligible_employees_shuffled = eligible_employees.sample(frac=1).reset_index(drop=True)
    np.random.seed(SEED) # 기타 NumPy 연산의 재현성 확보
    
    env_eval = RotationEnv(eligible_employees_shuffled, branches, distance_matrix, employees)
    obs = env_eval.reset()
    model = PPO.load("./models/rotation_ppo_model")

    # 3. 준비된 환경과 모델을 사용하여 이동 대상 직원들의 새로운 지점 배치를 순차적으로 예측
    assignments = []
    while True:
        action, _ = model.predict(obs) # deterministic=False (기본값)으로 정책에 따른 샘플링
        assignments.append(int(action))
        obs, _, done, _ = env_eval.step(action) # 환경 상태 업데이트
        if done: # 모든 직원이 배치되면 종료
            break

    # 4. 예측된 배치 결과를 바탕으로 최종 발령표(DataFrame)를 구성
    eligible_employees_shuffled['new_branch'] = [branch_names[a] for a in assignments]
    eligible_employees_shuffled['reassigned'] = True
    final_df = pd.concat([eligible_employees_shuffled, non_eligible_employees], ignore_index=True)\
    .sort_values(by='employee_id')

    # 5. 생성된 배치안이 사전에 정의된 운영상의 필수 조건들을 만족하는지 검증
    # 5-1. 지점 인원 수 조건 검증
    branch_counts = final_df['new_branch'].value_counts()
    if not branch_counts.between(8, 12).all():
        print(f"❌ 조건 불만족 (시도 {attempt}): 지점별 인원 범위(8~12명) 이탈")
        continue

    # 5-2. 4년 이상 근무자 이동 조건 검증
    moved_4yrs_subset = eligible_employees_shuffled[eligible_employees_shuffled['years_at_branch'] >= 4].copy()
    if not moved_4yrs_subset.empty:
        moved_4yrs_subset['moved'] = moved_4yrs_subset['current_branch'] != moved_4yrs_subset['new_branch']
        if not moved_4yrs_subset['moved'].all():
            print(f"❌ 조건 불만족 (시도 {attempt}): 4년 이상 근무자 미이동 발생")
            continue
    
    # 5-3. 평균 거리 조건 검증 (현재 로직: 새 평균 거리가 이전 평균보다 1 이상 크게 늘어나지 않아야 함)
    past_distances, current_distances = [], []
    branch_name_to_idx = {name: idx for idx, name in enumerate(branch_names)}
    for _, row in final_df.iterrows():
        home = np.array([row['home_x'], row['home_y']])
        past_idx = branch_name_to_idx[row['current_branch']]
        curr_idx = branch_name_to_idx[row['new_branch']]
        past_loc = branches.iloc[past_idx][['branch_x', 'branch_y']].values
        curr_loc = branches.iloc[curr_idx][['branch_x', 'branch_y']].values
        past_distances.append(np.linalg.norm(home - past_loc))
        current_distances.append(np.linalg.norm(home - curr_loc))
    past_mean = np.mean(past_distances) if past_distances else 0
    curr_mean = np.mean(current_distances) if current_distances else 0
    diff_metric = past_mean - curr_mean # 이 값이 양수면 거리 감소

    if diff_metric <= -1.0: # 현재 코드의 `if diff > -1.0: continue`의 실패 조건
        print(f"❌ 조건 불만족 (시도 {attempt}): 평균 거리 1 이상 증가 (변화량: {diff_metric:.2f})")
        continue

    # 6. 모든 검증 조건을 성공적으로 통과한 경우, 결과를 저장하고 루프 종료
    success = True
    filename = f"./data/rotation_assignments_final_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    final_df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n✅ 조건 충족 완료! (시도 {attempt}) 결과 저장: {filename}")
    print(f"▶ 지점별 인원: {branch_counts.to_dict()}")
    if not moved_4yrs_subset.empty:
         moved_percentage = 100.0 if moved_4yrs_subset['moved'].all() else (moved_4yrs_subset['moved'].sum() / len(moved_4yrs_subset) * 100)
         print(f"▶ 4년 이상 근무자 수(이동 대상): {len(moved_4yrs_subset)}명, 이동률: {moved_percentage:.2f}%")
    else:
        print(f"▶ 4년 이상 근무자 수(이동 대상): 0명")
    print(f"▶ 평균 거리 변화 (이전-현재): {diff_metric:.2f} (양수면 거리 감소)")
    break

# 7. 최대 시도 횟수 내에 모든 조건을 만족하는 해를 찾지 못한 경우, 최종 실패 메시지 출력
if not success:
    print(f"\n⚠️ 최대 반복 횟수({max_attempts})에 도달했지만 모든 조건을 충족하는 결과를 찾지 못했습니다.")

1, ❌ 조건 불만족 (시도 1): 지점별 인원 범위(8~12명) 이탈
2, ❌ 조건 불만족 (시도 2): 지점별 인원 범위(8~12명) 이탈
3, ❌ 조건 불만족 (시도 3): 지점별 인원 범위(8~12명) 이탈
4, ❌ 조건 불만족 (시도 4): 지점별 인원 범위(8~12명) 이탈
5, ❌ 조건 불만족 (시도 5): 지점별 인원 범위(8~12명) 이탈
6, ❌ 조건 불만족 (시도 6): 지점별 인원 범위(8~12명) 이탈
7, ❌ 조건 불만족 (시도 7): 지점별 인원 범위(8~12명) 이탈
8, ❌ 조건 불만족 (시도 8): 지점별 인원 범위(8~12명) 이탈
9, ❌ 조건 불만족 (시도 9): 지점별 인원 범위(8~12명) 이탈
10, ❌ 조건 불만족 (시도 10): 지점별 인원 범위(8~12명) 이탈
11, ❌ 조건 불만족 (시도 11): 지점별 인원 범위(8~12명) 이탈
12, ❌ 조건 불만족 (시도 12): 지점별 인원 범위(8~12명) 이탈
13, ❌ 조건 불만족 (시도 13): 지점별 인원 범위(8~12명) 이탈
14, ❌ 조건 불만족 (시도 14): 지점별 인원 범위(8~12명) 이탈
15, ❌ 조건 불만족 (시도 15): 지점별 인원 범위(8~12명) 이탈
16, ❌ 조건 불만족 (시도 16): 지점별 인원 범위(8~12명) 이탈
17, ❌ 조건 불만족 (시도 17): 지점별 인원 범위(8~12명) 이탈
18, ❌ 조건 불만족 (시도 18): 지점별 인원 범위(8~12명) 이탈
19, ❌ 조건 불만족 (시도 19): 지점별 인원 범위(8~12명) 이탈
20, ❌ 조건 불만족 (시도 20): 지점별 인원 범위(8~12명) 이탈
21, ❌ 조건 불만족 (시도 21): 지점별 인원 범위(8~12명) 이탈
22, ❌ 조건 불만족 (시도 22): 지점별 인원 범위(8~12명) 이탈
23, ❌ 조건 불만족 (시도 23): 지점별 인원 범위(8~12명) 이탈
24, ❌ 조건 불만족 (시도 24): 지점별 인원 범위(8~12명) 이탈
25, ❌ 조건 불